# ![image-alt-text](https://raw.githubusercontent.com/FabricTools/fabric-icons/refs/heads/main/node_modules/%40fabric-msft/svg-icons/dist/svg/notebook_64_item.svg) 
# DAX Query x Semantic Link Labs x DAX INFO Functions 

##### Purpose of notebook is to collect, analyze and store semantic model INFO functions and DAX Queries in a Lakehouse and share with semantic model or report owners or **_"crazy solution documenters"_**
_This can be run in a Python notebook_

## Credit to following
- [Michael Kovalsky - Semantic Link Labs](https://github.com/microsoft/semantic-link-labs/tree/main/notebooks)
- [List of all DAX INFO functions](https://learn.microsoft.com/en-us/dax/info-functions-dax)
- [Fourmoo on Python Notebook advantages vs spark notebooks](https://www.fourmoo.com/2025/04/23/what-are-the-advantages-of-using-python-notebook-vs-a-spark-notebook-in-microsoft-fabric/)
- [Hariharan Rajendran's DAX Query on Models Issues and Model Summary](https://www.linkedin.com/in/imhariharanr/)
- [Power BI Tips on Save to Delta table with Python](https://www.youtube.com/watch?v=IUsGE4pS9VU-URL)
- [Reza Rad's take on Power BI model analysis using DAX INFO functions ](https://radacad.com/power-bi-model-analysis-using-dax-info-functions/)
- [David Kofod Hanna's GitHub on Power BI Documentation System](https://github.com/DKH-DK/Self-Service-Power-BI-Fabric/tree/main/Power%20BI%20Documentation%20System)


#### If you feel frustrated like me, not all DAX Info functions can be used like INFO.VIEW functions by adding a calculated DAX table.

#### Then we have been forced to used DAX Query View.

![image-alt-text](https://i0.wp.com/radacad.com/wp-content/uploads/2025/06/image-8.png?w=1325&ssl=1)

#### **Save your frustration**, because I found a way to store DAX Query results back to a lakehouse 🤓


### Configurations

In [ ]:
# Install Semantic Link Labs
%pip install semantic-link-labs

# Import packages and name alias
import pandas as pd
from datetime import datetime
import sempy_labs as labs
from sempy_labs.tom import connect_semantic_model
import sempy.fabric as fabric


In [ ]:
#Configuration for Workspace and Semantic model and Lakehouse
WorkspaceName = "Fabric Analyst in A Day"
SemanticModelName = "Sales Semantic Model"
LakehouseName = "LakehouseDocumentationINFODAX"

### Manual DAX Query

In [ ]:
# Define DAX Query to execute
dax_query = """
 EVALUATE
 INFO.TABLES()
 """

In [ ]:
# Run DAX Query against Semantic Model
df_result_manual = fabric.evaluate_dax(
    dataset=SemanticModelName, 
    dax_string=dax_query, 
    workspace=WorkspaceName
    )
    
# Display the result
display(df_result_manual)

In [ ]:
labs.save_as_delta_table(
    dataframe=df_result_manual,
    delta_table_name="daxquery",
    write_mode="overwrite",   # or "append" based on your requirement
    merge_schema=False,       # Set to True if you want to merge schema
    schema=None,              # Provide schema if needed
    lakehouse=LakehouseName, 
    workspace=None            
)

### Automated DAX Query loop and save to delta table

In [ ]:
# Defining DF of all INFO DAX Functions and descriptions

dataDAX = [
    ("INFO.ALTERNATEOFDEFINITIONS()", ""),
    ("INFO.ANNOTATIONS()", "Returns a list of all annotations in the current model with columns matching the schema rowset for annotation objects."),
    ("INFO.ATTRIBUTEHIERARCHIES()", "Represents the TMSCHEMA_ATTRIBUTE_HIERARCHIES DMV query function."),
    ("INFO.ATTRIBUTEHIERARCHYSTORAGES()", ""),
    ("INFO.CALCDEPENDENCY()", "Returns information about the calculation dependency for a DAX query."),
    ("INFO.CALCULATIONGROUPS()", ""),
    ("INFO.CALCULATIONITEMS()", ""),
    ("INFO.CATALOGS()", "Represents the DBSCHEMA_CATALOGS DMV query function."),
    ("INFO.CHANGEDPROPERTIES()", "Represents the TMSCHEMA_CHANGED_PROPERTIES DMV query function."),
    ("INFO.COLUMNPARTITIONSTORAGES()", ""),
    ("INFO.COLUMNPERMISSIONS()", "Returns a list of all column permissions in the current model with columns matching the schema rowset for column permissions objects."),
    ("INFO.COLUMNS()", "Returns a list of all columns in the current model with columns matching the schema rowset for column objects."),
    ("INFO.COLUMNSTORAGES()", "Returns a list of all column storages in the current model with columns matching the schema rowset for column storage objects."),
    ("INFO.CSDLMETADATA()", "Returns information about database metadata in XML format."),
    ("INFO.CULTURES()", "Returns a list of all cultures in the current model with columns matching the schema rowset for culture objects."),
    ("INFO.DATACOVERAGEDEFINITIONS()", ""),
    ("INFO.DATASOURCES()", "Represents the TMSCHEMA_DATASOURCES DMV query function."),
    ("INFO.DELTATABLEMETADATASTORAGES()", ""),
    ("INFO.DEPENDENCIES()", "Returns information about the calculation dependency for a DAX query."),
    ("INFO.DETAILROWSDEFINITIONS()", "Returns a list of all detail rows definitions in the current model with columns matching the schema rowset for detail rows definitions objects."),
    ("INFO.DICTIONARYSTORAGES()", ""),
    ("INFO.EXCLUDEDARTIFACTS()", "Represents the TMSCHEMA_EXCLUDED_ARTIFACTS DMV query function."),
    ("INFO.EXPRESSIONS()", "Returns a list of all expressions in the current model with columns matching the schema rowset for expressions objects."),
    ("INFO.EXTENDEDPROPERTIES()", "Returns a list of all extended properties in the current model with columns matching the schema rowset for extended properties objects."),
    ("INFO.FORMATSTRINGDEFINITIONS()", ""),
    ("INFO.FUNCTIONS()", "Returns information about the functions that are currently available for use in the DAX programming language."),
    ("INFO.GENERALSEGMENTMAPSEGMENTMETADATASTORAGES()", ""),
    ("INFO.GROUPBYCOLUMNS()", ""),
    ("INFO.HIERARCHIES()", "Represents the TMSCHEMA_HIERARCHIES DMV query function."),
    ("INFO.HIERARCHYSTORAGES()", ""),
    ("INFO.KPIS()", "Returns a list of all KPIS in the current model with columns matching the schema rowset for KPI objects."),
    ("INFO.LEVELS()", "Returns a list of all levels in the current model with columns matching the schema rowset for level objects."),
    ("INFO.LINGUISTICMETADATA()", "Represents the TMSCHEMA_LINGUISTIC_METADATA DMV query function."),
    ("INFO.MEASURES()", "Returns a list of all measures in the current model with columns matching the schema rowset for measure objects."),
    ("INFO.MODEL()", "Represents the TMSCHEMA_MODEL DMV query function."),
    ("INFO.OBJECTTRANSLATIONS()", "Returns a list of all object translations in the current model with columns matching the schema rowset for object translation objects."),
    ("INFO.PARQUETFILESTORAGES()", ""),
    ("INFO.PARTITIONS()", "Represents the TMSCHEMA_PARTITIONS DMV query function."),
    ("INFO.PARTITIONSTORAGES()", "Returns a list of all partition storages in the current model with columns matching the schema rowset for partition storage objects."),
    ("INFO.PERSPECTIVECOLUMNS()", "Returns a list of all perspective columns in the current model with columns matching the schema rowset for perspective columns objects."),
    ("INFO.PERSPECTIVEHIERARCHIES()", "Returns a list of all perspective hierarchies in the current model with columns matching the schema rowset for perspective hierarchies objects."),
    ("INFO.PERSPECTIVEMEASURES()", "Returns a list of all perspective measures in the current model with columns matching the schema rowset for perspective measures objects."),
    ("INFO.PERSPECTIVES()", "Returns a list of all perspectives in the current model with columns matching the schema rowset for perspectives objects."),
    ("INFO.PERSPECTIVETABLES()", "Returns a list of all perspective tables in the current model with columns matching the schema rowset for perspective tables objects."),
    ("INFO.PROPERTIES()", "Represents the DISCOVER_PROPERTIES DMV query function."),
    ("INFO.QUERYGROUPS()", ""),
    ("INFO.REFRESHPOLICIES()", ""),
    ("INFO.RELATEDCOLUMNDETAILS()", ""),
    ("INFO.RELATIONSHIPINDEXSTORAGES()", ""),
    ("INFO.RELATIONSHIPS()", "Represents the TMSCHEMA_RELATIONSHIPS DMV query function."),
    ("INFO.RELATIONSHIPSTORAGES()", ""),
    ("INFO.ROLEMEMBERSHIPS()", "Returns a list of all role memberships in the current model with columns matching the schema rowset for role memberships objects."),
    ("INFO.ROLES()", "Returns a list of all roles in the current model with columns matching the schema rowset for roles objects."),
    ("INFO.SEGMENTMAPSTORAGES()", "Returns a list of all segment map storages in the current model with columns matching the schema rowset for segment map storage objects."),
    ("INFO.SEGMENTSTORAGES()", ""),
    ("INFO.STORAGEFILES()", "Returns a list of all storage files in the current model with columns matching the schema rowset for storage file objects."),
    ("INFO.STORAGEFOLDERS()", "Returns a list of all storage folders in the current model with columns matching the schema rowset for storage folder objects."),
    ("INFO.STORAGETABLECOLUMNS()", "Returns statistics about the columns of in-memory tables."),
    ("INFO.STORAGETABLECOLUMNSEGMENTS()", "Returns information about the column segments used for storing data for in-memory tables."),
    ("INFO.STORAGETABLES()", "Returns statistics about in-memory tables."),
    ("INFO.TABLEPERMISSIONS()", "Returns a list of all table permissions in the current model with columns matching the schema rowset for table permissions objects."),
    ("INFO.TABLES()", "Returns a list of all tables in the current model with columns matching the schema rowset for table objects."),
    ("INFO.TABLESTORAGES()", "Returns a list of all table storages in the current model with columns matching the schema rowset for table storage objects."),
    ("INFO.VARIATIONS()", "Returns a list of all variations in the current model with columns matching the schema rowset for variations objects."),
    ("INFO.VIEW.COLUMNS()", "Returns a list of all columns in the current model."),
    ("INFO.VIEW.MEASURES()", "Returns a list of all measures in the current model."),
    ("INFO.VIEW.RELATIONSHIPS()", "Returns a list of all relationships in the current model."),
    ("INFO.VIEW.TABLES()", "Returns a list of all tables in the current model."),
]

In [ ]:
# Loop through each DAX Info function and save as table

# List of INFO DAX Functions: (Function call, Description) - see here for all: https://learn.microsoft.com/en-us/dax/info-functions-dax
data = dataDAX

df_info = pd.DataFrame(data, columns=["Function", "Description"])

# Current timestamp
timestamp = datetime.utcnow().isoformat()


# Loop through each function in line 5
for func_name, description in data:
    print(f"Processing: {func_name}")

    dax_query = f"""
    EVALUATE
    {func_name}
    """

    try:
        # Run the DAX Query on the 
        df_result = fabric.evaluate_dax(
            dataset=SemanticModelName,
            dax_string=dax_query,
            workspace=WorkspaceName
        )

        # Add metadata columns
        df_result["query_timestamp"] = timestamp
        df_result["workspace"] = WorkspaceName
        df_result["semantic_model"] = SemanticModelName

        # Clean up function name to use as table name
        table_name = func_name.replace("INFO.", "").replace("()", "").replace(".", "_").lower()

        # Save result as a Delta table
        labs.save_as_delta_table(
            dataframe=df_result,
            delta_table_name=table_name,
            write_mode="overwrite",         # or "append" based on your requirement
            merge_schema=False,             # Set to True if you want to merge schema
            schema=None,                    # Provide schema if needed
            lakehouse=LakehouseName,
            workspace=None
        )
        
        print(f"Saved table: {table_name}")

    except Exception as e:
        print(f"Failed processing {func_name}: {str(e)}")